## Idea

## Reward Analysis

## Ablation

## Do we have to soup?

## KL-Hack

In [13]:
#| code-summary: Imports and model evaluation function
#| output: false
#| echo: false
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, functools, itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from einops import rearrange

from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer

for d in ['data', 'models', 'logs']: os.makedirs(d, exist_ok = True)

device = torch.device(
    'cuda' if torch.cuda.is_available() else
    ('mps' if torch.backends.mps.is_available() else
    'cpu')
)

def set_seed(seed = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device.type == 'cuda':
        print('Setting seed for CUDA')
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()
device

device(type='mps')

In [16]:
#|code-summary: Get model and SFT data

def format_chat(example):
    """Format UltraChat example into chat format"""
    conversation = f"<start_of_turn>user\n{example['prompt']}\n<end_of_turn>\n"
    
    for msg in example['messages']:
        conversation += f"<start_of_turn>{msg['role']}\n{msg['content']}\n<end_of_turn>\n"
    
    return {"text": conversation.strip()}

model_name = 'HuggingFaceTB/SmolLM2-135M'
get_model = lambda: AutoModelForCausalLM.from_pretrained(model_name).to(device)
model = get_model()
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

sft_data = load_dataset('HuggingFaceH4/ultrachat_200k')
format_dataset = lambda dataset: dataset.map(format_chat, remove_columns = dataset.column_names)
sft_train, sft_test = format_dataset(sft_data['train_sft']), format_dataset(sft_data['test_sft'])

In [17]:
sft_config = SFTConfig(
    dataset_text_field = 'text',
    learning_rate = 2e-5,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 16,
    max_seq_length = 2048,
    output_dir = 'models/sft',
    bf16 = True,
    packing = True,
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model = model,
    train_dataset = sft_train,
    eval_dataset = sft_test,
    args = sft_config,
)

trainer.train()

  0%|          | 0/57642 [00:00<?, ?it/s]


KeyboardInterrupt

